## Star Tracker ML Pipeline ##

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

print("TensorFlow version:", tf.__version__)

2024-07-03 16:37:56.147069: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 16:37:57.260483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.16.1


### Filepaths and Hyperparameters ###

In [2]:
# Data filepath for training
file = "../images_data/mag5_1608_47deg_gray/bins/mag5_1608_47deg_bin_features.csv"          # <-- Configure together
file_descript = "mag5_1608_47deg"     # Magnitude of stars, num classes, camera fov         # <-- Configure together

# Configuration values
validation_split = 0.20
num_epochs = 60
batch_size = 40
saved_model_dir = 'models/'

In [3]:

# TODO: read the header instead of hardcoding it
dataframe = pd.read_csv(file, header=0)
input_size = len(dataframe.columns) - 1
dataset = dataframe.values
X = dataset[:,1:(input_size+1)].astype(int)
Y = dataset[:,0].astype(int)
print("Input Size:", len(X[0]))
num_classes = len(np.unique(Y))
print("Output Size:", num_classes)

Input Size: 10
Output Size: 1608


In [4]:
# Encode class values as integers and then create dataframe with one hot encoding
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_Y = tf.one_hot(encoded_Y, depth=num_classes)
print(dummy_Y)



# Unused code from when I was sanity checking
"""
for i in range(100):
    if not (encoder.classes_[dummy_Y.numpy()[i].argmax()] == Y[i]):
        print("Error")
        # print(dummy_Y.numpy()[i].argmax())
        # print(Y[i])
        # print(encoder.classes_[dummy_Y.numpy()[i].argmax()])
"""

"""
# star_labels = np.zeros(num_classes).astype(int)
# get labels in num_classes sized list

# one_hot_array = dummy_Y.numpy()
# # print(Y[0])
# for i in range(len(one_hot_array)):
#     if star_labels[one_hot_array[i].argmax()] == 0:
#         star_labels[one_hot_array[i].argmax()] = Y[i]
# np.set_printoptions(threshold=np.inf)
# print(star_labels)
"""


2024-07-03 16:38:00.075175: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 16:38:00.484594: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 16:38:00.484672: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 16:38:00.501777: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(65928, 1608), dtype=float32)


2024-07-03 16:38:00.501902: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 16:38:00.501950: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 16:38:01.611943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 16:38:01.612009: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 16:38:01.612018: I tensorflow/core/common_runtime/gpu/gpu

'\n# star_labels = np.zeros(num_classes).astype(int)\n# get labels in num_classes sized list\n\n# one_hot_array = dummy_Y.numpy()\n# # print(Y[0])\n# for i in range(len(one_hot_array)):\n#     if star_labels[one_hot_array[i].argmax()] == 0:\n#         star_labels[one_hot_array[i].argmax()] = Y[i]\n# np.set_printoptions(threshold=np.inf)\n# print(star_labels)\n'

In [5]:
# Create the ML Model
star_model = tf.keras.Sequential([
  layers.Dense(input_size, activation="relu", name="layer1"),
  layers.Dense(64, activation="relu", name="layer2"),
  layers.Dense(32, activation="relu", name="layer3"),
  layers.Dense(1608, activation="softmax", name ="layer4")
])

star_model.compile(loss = "categorical_crossentropy",
                      optimizer = 'adam', metrics = ['accuracy'])

In [6]:
# Model Training
star_model.fit(X, dummy_Y, epochs=num_epochs)

Epoch 1/60


I0000 00:00:1720049882.901514  129220 service.cc:145] XLA service 0x7f3538004e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720049882.901599  129220 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-07-03 16:38:02.933693: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-03 16:38:03.112387: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720049884.304134  129275 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 48 bytes spill stores, 48 bytes spill loads

I0000 00:00:1720049884.330152  129273 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 304 bytes spill stores, 304 bytes spill loads


 106/2061 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0018 - loss: 7.3792

I0000 00:00:1720049885.251217  129220 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2045/2061 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0853 - loss: 5.3296

I0000 00:00:1720049888.487771  129400 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 60 bytes spill stores, 60 bytes spill loads



2061/2061 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.0862 - loss: 5.3168
Epoch 2/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5024 - loss: 1.6869
Epoch 3/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7234 - loss: 0.9074
Epoch 4/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8065 - loss: 0.6313
Epoch 5/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8443 - loss: 0.5117
Epoch 6/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8631 - loss: 0.4515
Epoch 7/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8828 - loss: 0.3850
Epoch 8/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8922 - loss: 0.3583
Epoch 9/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8994 - loss: 0.3294
Epoch 10/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9036 - loss: 0.3171
Epoch 11/60
2061/2061 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9111 - loss: 0.2935
Epoch 12/60
2061/2061 ━━━━━━━━━━━━━━━━━━━

In [8]:
# Save Model

star_model.save(saved_model_dir + file_descript +'.keras')
if not os.path.exists(saved_model_dir + file_descript +'_saved_model/'):
    os.makedirs(saved_model_dir + file_descript +'_saved_model/')
star_model.export(saved_model_dir + file_descript +'_saved_model/')

# TODO: tflite conversion is not working.
# # Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir + file_descript +'_saved_model/') # path to the SavedModel directory
tflite_model = converter.convert()

# # Save the model.
with open(saved_model_dir + file_descript + '_small.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: models/mag5_1608_47deg_saved_model/assets


INFO:tensorflow:Assets written to: models/mag5_1608_47deg_saved_model/assets


Saved artifact at 'models/mag5_1608_47deg_saved_model/'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1608), dtype=tf.float32, name=None)
Captures:
  139871074915552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139871074923824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139871074912384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139871059007920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139871059005104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139871059017424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139871074917664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139871059271472: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1720050176.203430  129083 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1720050176.203467  129083 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-07-03 16:42:56.203661: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/mag5_1608_47deg_saved_model/
2024-07-03 16:42:56.203993: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-03 16:42:56.204003: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: models/mag5_1608_47deg_saved_model/
2024-07-03 16:42:56.208258: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-07-03 16:42:56.227455: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: models/mag5_1608_47deg_saved_model/
2024-07-03 16:42:56.233408: I tensorflow/cc/saved_model/loader.cc:317] SavedModel load for tags { serve }; Status: success: OK. Took 29750 mi